# Random Forests Notebook
[Return to project overview](final_project_overview.ipynb),

### Andrew Larimer, Deepak Nagaraj, Daniel Olmstead, Michael Winton (W207-4-Summer 2018 Final Project)

### Importing Libraries and setting options

First we import necessary libraries, including our util functions, and set Pandas and Matplotlib options.

In [1]:
# import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import graphviz

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedStratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from util import our_train_test_split, read_data, get_dummies, \
    print_cv_results, run_model_get_ordered_predictions, create_passnyc_list
import pickle

# set default options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
%matplotlib inline

### Reading our data using util cleanup and imputing

Our util module has shared utility functions for cleaning up our data and imputing means.

In [2]:
# Read the cleaned, merged, and mean-imputed data from our utility function
train_data, test_data, train_labels, test_labels = read_data(do_imputation=True)

Train: 371 observations (positive class fraction: 0.232)
Test : 93 observations (positive class fraction: 0.226)


We'll drop some columns that were used to calculate our dependendt variable, as well as our index column, school name strings, and `school_income_estimate` which had too many missing values to fill via imputing.

In [3]:
# We drop a few features for the following reasons:
#    Used in generating dependent variable: 'num_shsat_test_takers',
#        'offers_per_student', 'pct_test_takers'
#    Strings or other non-features: 'dbn', 'school_name'
#    Too many empty values: 'school_income_estimate'
#    Data preserved in other features: 'zip', 'rigorous_instruction_rating',
#       'collaborative_teachers_rating', 'supportive_environment_rating',
#       'effective_school_leadership_rating',
#       'strong_family_community_ties_rating', 'trust_rating'
#    Found not to help model: 'district' (or one-hot encoding)

FEATURES_TO_DROP = ['dbn', 'school_name', 'zip', 'num_shsat_test_takers',
                    'offers_per_student', 'pct_test_takers', 'school_income_estimate',
                    'rigorous_instruction_rating','collaborative_teachers_rating',
                    'supportive_environment_rating',
                    'effective_school_leadership_rating',
                    'strong_family_community_ties_rating', 'trust_rating',
                    'district']

# We'll go ahead and drop total_columns_to_drop columns.
train_prepped = train_data.drop(FEATURES_TO_DROP,axis=1)
test_prepped = test_data.drop(FEATURES_TO_DROP,axis=1)

In [4]:
# We confirm our resulting data has no more NAs
print("Confirm total of remaining NAs is: ",np.sum(np.sum(train_prepped.isna())))

Confirm total of remaining NAs is:  0


### Optimizing a Random Forest Model on Cross-Validation

We now move into training our random forest model. To optimize our hyperparameter of how many trees to include in our forest, we use GridSearchCV and take advantage of its cross validation capability to use cross validation against our training set instead of further reducing our data into smaller train and dev sets. 

In [5]:
# We define into how many groups we'd like to split our test data
# for use in cross-validation to evaluate hyperparameters.
KFOLDS = 5

# We check for previously saved results and a serialized model saved
# to disk before re-running GridSearchCV. To force it to run again,
# we can comment out the try: & except: or just delete the last saved
# results.

try:

    cv_results = pd.read_csv('cache_forest/forest_gridsearch_results.csv')
    with open('cache_forest/pickled_forest','rb') as f:
        forest_cv = pickle.load(f)

except:

    # If no saved results are found, we define our base Random Forest
    # Classifier with fixed parameters we don't anticipate adjusting.
    # We want to run as many jobs as we have cores at once (which is
    # what the -1 input to n_jobs does, and we define our random state
    # for reproducibility.)
    forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced',
                                    random_state=207)

    # We define a range of paramters we'd like to try for our forest.
    params_to_try = {'n_estimators':[10,30,100,300],
                     'max_depth':[None,2,5,7],
                     'min_samples_leaf':[1,2,4],
                     'min_samples_split':[2,3,5],
                     'max_features':[.2,.5,.8]}

    # Now we run GridSearchCV on our forest estimator, trying our varying
    # numbers of trees and utilizing our cross validation to determine the
    # best number of trees across the best number of train/dev cross
    # validation splits, using a weighted F1 score as our metric of success.
    forest_cv = GridSearchCV(forest, params_to_try, scoring=['f1',
                            'accuracy'], refit='f1', cv=KFOLDS,
                             return_train_score=False)

    # We'll time it and report how long it took to run:
    start_time = time.time()
    forest_cv.fit(train_prepped, train_labels)
    end_time = time.time()
    
    took = int(end_time - start_time)
    print("Grid search took {0:d} minutes, {1:d} seconds.".format(
              took // 60, took % 60))

    # And pickle our trained model, and save our scores to csv.
    with open('cache_forest/pickled_forest','wb') as f:
        pickle.dump(forest_cv, f)

    cv_results = pd.DataFrame(forest_cv.cv_results_)
    cv_results.to_csv('cache_forest/forest_gridsearch_results.csv')
    
# Then display our results in a Pandas dataframe, sorted by
# rank based on mean f1 score across 5-fold CV testing:
cv_results.sort_values('rank_test_f1')

,Unnamed: 0,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
266,266,0.165147,0.001103,0.212079,0.002194,5.0,0.5,2,2,100,"{'max_depth': 5, 'max_features': 0.5, 'min_sam...",0.787879,0.666667,0.666667,0.578947,0.731707,0.686647,0.070353,1,0.906667,0.864865,0.824324,0.783784,0.851351,0.846361,0.041066,26
270,270,0.164223,0.001474,0.211624,0.002179,5.0,0.5,2,3,100,"{'max_depth': 5, 'max_features': 0.5, 'min_sam...",0.787879,0.666667,0.666667,0.578947,0.731707,0.686647,0.070353,1,0.906667,0.864865,0.824324,0.783784,0.851351,0.846361,0.041066,26
322,322,0.165117,0.001801,0.214894,0.002080,5.0,0.8,4,5,100,"{'max_depth': 5, 'max_features': 0.8, 'min_sam...",0.787879,0.666667,0.666667,0.615385,0.681818,0.683964,0.056932,3,0.906667,0.851351,0.824324,0.797297,0.810811,0.838275,0.038780,87
314,314,0.163819,0.001699,0.211772,0.002795,5.0,0.8,4,2,100,"{'max_depth': 5, 'max_features': 0.8, 'min_sam...",0.787879,0.666667,0.666667,0.615385,0.681818,0.683964,0.056932,3,0.906667,0.851351,0.824324,0.797297,0.810811,0.838275,0.038780,87
318,318,0.163367,0.001009,0.213975,0.002829,5.0,0.8,4,3,100,"{'max_depth': 5, 'max_features': 0.8, 'min_sam...",0.787879,0.666667,0.666667,0.615385,0.681818,0.683964,0.056932,3,0.906667,0.851351,0.824324,0.797297,0.810811,0.838275,0.038780,87
310,310,0.170330,0.006534,0.212620,0.001523,5.0,0.8,2,5,100,"{'max_depth': 5, 'max_features': 0.8, 'min_sam...",0.787879,0.687500,0.648649,0.578947,0.714286,0.683734,0.069407,6,0.906667,0.864865,0.824324,0.783784,0.837838,0.843666,0.041093,40
170,170,0.166698,0.001896,0.210950,0.002953,2.0,0.5,4,2,100,"{'max_depth': 2, 'max_features': 0.5, 'min_sam...",0.800000,0.722222,0.682927,0.585366,0.625000,0.683418,0.075142,7,0.906667,0.864865,0.824324,0.770270,0.756757,0.824798,0.056495,283
174,174,0.166968,0.001583,0.210940,0.002204,2.0,0.5,4,3,100,"{'max_depth': 2, 'max_features': 0.5, 'min_sam...",0.800000,0.722222,0.682927,0.585366,0.625000,0.683418,0.075142,7,0.906667,0.864865,0.824324,0.770270,0.756757,0.824798,0.056495,283
178,178,0.166740,0.000333,0.210377,0.000891,2.0,0.5,4,5,100,"{'max_depth': 2, 'max_features': 0.5, 'min_sam...",0.800000,0.722222,0.682927,0.585366,0.625000,0.683418,0.075142,7,0.906667,0.864865,0.824324,0.770270,0.756757,0.824798,0.056495,283
274,274,0.164173,0.001499,0.213329,0.001854,5.0,0.5,2,5,100,"{'max_depth': 5, 'max_features': 0.5, 'min_sam...",0.787879,0.666667,0.666667,0.578947,0.714286,0.683172,0.068446,10,0.906667,0.864865,0.824324,0.783784,0.837838,0.843666,0.041093,40


In [6]:
# We extract our best model and best parameters from our GridSearchCV results.
best_forest = forest_cv.best_estimator_
best_params = forest_cv.best_params_

# We reiterate our preferred number of cross validation folds if we haven't
# had to re-train our model
KFOLDS = 5

print("Best params:\n")
for param, val in best_params.items():
    print(param,':',val)

print("\n")

winning_cv_results = cv_results[cv_results['rank_test_f1'] == 1].iloc[1,:]

# display accuracy with 95% confidence interval
winning_mean_accuracy = winning_cv_results['mean_test_accuracy']
std_accuracy = winning_cv_results['std_test_accuracy']
print('With %d-fold cross-validation,\nAccuracy is: %.3f (95%% CI from %.3f to %.3f).' %
          (KFOLDS, winning_mean_accuracy,
           float(winning_mean_accuracy - 1.96 * std_accuracy),
           float(winning_mean_accuracy + 1.96 * std_accuracy)))

# display F1 score with 95% confidence interval
winning_mean_f1 = winning_cv_results['mean_test_f1']
std_f1 = winning_cv_results['std_test_f1']
print('The F1 score is: %.3f (95%% CI from %.3f to %.3f).' %
          (winning_mean_f1,
           float(winning_mean_f1 - 1.96 * std_f1),
           float(winning_mean_f1 + 1.96 * std_f1)))

Best params:

max_depth : 5
max_features : 0.5
min_samples_leaf : 2
min_samples_split : 2
n_estimators : 100


With 5-fold cross-validation,
Accuracy is: 0.846 (95% CI from 0.766 to 0.927).
The F1 score is: 0.687 (95% CI from 0.549 to 0.825).


### Analyzing our Top 10 Features
These features have the highest feature importance scores as found by our best forest model.

Unsurprisingly, they tend to include our most general metrics of performance, like average proficiency and grade 7 ela scores of 4 across all students.

It is interesting to see how heavily the absence and attendance rates factor in.

In [7]:
features = train_prepped.columns
feature_importances = best_forest.feature_importances_

features_and_importances = pd.DataFrame(feature_importances,features,['Importances'])

# Need column names here after the ohe_data step to analyze the results
features_and_importances.sort_values('Importances', ascending=False).iloc[1:11,]

,Importances
average_math_proficiency,0.175213
grade_7_ela_4s_all_students,0.118439
grade_7_math_4s_all_students,0.099211
percent_of_students_chronically_absent,0.045525
percent_hispanic,0.034358
student_attendance_rate,0.026382
school_pupil_teacher_ratio,0.025499
average_class_size_english,0.018679
average_class_size_social_studies,0.015125
strong_family_community_ties_percent,0.014503


### Viewing a few trees from our forest

To see what decisions some of our trees are coming to, let's take a look at three random trees out of our group of estimators.

In [8]:
trees_in_forest = best_forest.estimators_
max_index = len(trees_in_forest)

random_indeces = np.random.randint(0, max_index, 3)
example_graphs = []

for index in random_indeces:
    tree_viz = export_graphviz(trees_in_forest[index], proportion=True, filled=True,
                               feature_names=train_prepped.columns, rounded=True,
                               class_names=['not_high_registrations','high_registrations'],
                               out_file=None)
    try:
        graphviz.Source(source=tree_viz,
                        filename='cache_forest/tree_viz_{0}'.format(index),
                        format='svg').render()
    except ExecutableNotFound:
        print("Your system lacks GraphViz. Instructions to install for your" + \
            "operating system should be available at https://graphviz.gitlab.io/download/" + \
            "The images will be loaded and linked to below, so you don't need it to view" + \
            "this notebook.")

In the displayed graphs below, the more orange a cell is, the more the samples that pass through it tend to be not in our "high_registrations" category. The more blue a cell is, the more it tends to include "high_registrations." We are using the gini measurement of impurity to structure our trees.

The samples percentage tells us what percentage of our total samples pass through this node.

The value list tells us how many of the samples that have reached this node are in each class. So the first value (value[0]) indicates what proportion of the samples in the node are not high_registrations, and the second value (value[1]) tells us how many are high_registrations. You can see that these values correspond to the coloring of the graph.

Then from each node, if a sample meets the condition that titles the node, it travels to the lower left branch. If it does not meed the condition of the node, it travels down the right branch.

#### Graph of Tree #13

![Graph #13](cache_forest/tree_viz_13.svg)

[Link to Graph #13 if not rendering on GitHub](https://www.dropbox.com/s/vqg9hm8ol2kxy7d/tree_viz_13.svg?dl=0)

#### Graph of Tree #39

![Graph #39](cache_forest/tree_viz_39.svg)

[Link to Graph #39 if not rendering on GitHub](https://www.dropbox.com/s/x0ny1fpk13yj16c/tree_viz_39.svg?dl=0)

#### Graph of Tree #77

![Graph #39](cache_forest/tree_viz_77.svg)

[Link to Graph #77 if not rendering on GitHub](https://www.dropbox.com/s/rlspal912qu0euf/tree_viz_77.svg?dl=0)

Remember these are just three out of our total 100 trees that make our ensemble predictor, and each of these trees only have half of the total features in our set. Their variation is what helps 'smooth out the edges' of some of the predictions, to gain the benefits of an ensemble within a single model.

All in all, the graph results are to be expected given the features that we found to be important, but the PASSNYC team specifically asked for models that could be explained, and we feel these trees would of course help explain the model's decision-making process clearly to all stakeholders.

### Measuring results on the test set

Now that we have determined our best preprocessing steps and hyperparameters,
we evaluate our results on our test set.

In [9]:
# We train on our full training data on a new forest with our best_params
# determined by our GridSearchCV
best_forest.fit(train_prepped, train_labels)
predictions = best_forest.predict(test_prepped)

# And make predictions on our test data
# predictions = best_forest.predict(test_prepped)
f1 = f1_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)
accuracy = np.sum(predictions == test_labels) / len(test_labels)
    
print("Average F1 Score: {0:.4f}".format(f1))
print("Accuracy: {0:.4f}".format(accuracy))

Average F1 Score: 0.7083
Accuracy: 0.8495


### Recommendations Based on Opportunity for Engaging with Black/Hispanic Student Populations

We will make our final recommendations based on the ranking methods described in our [overview notebook](final_project_overview.ipynb) that seek to identify the greatest opportunities for increasing SHSAT registrations at schools with high black and hispanic populations.

In [10]:
fp_df = run_model_get_ordered_predictions(best_forest, train_data, test_data,
                                      train_prepped, test_prepped,
                                      train_labels, test_labels)


# We now use another util function to generate the list we'll feed to our final
# ensemble evaluation.

df_passnyc = create_passnyc_list(fp_df, train_data, test_data, train_labels, test_labels)
# Write to CSV
df_passnyc.to_csv('results/results.randomforest.csv')

df_passnyc

,rank,1s,dbn,school_name,economic_need_index,grade_7_enrollment,num_shsat_test_takers,pct_test_takers,percent_black__hispanic,minority_delta,score
107,1,10.0,22K278,J.H.S. 278 MARINE PARK,0.494,428.0,98.0,33.0,63.0,76,76.0
56,2,5.0,27Q210,J.H.S. 210 ELIZABETH BLACKWELL,0.602,643.0,169.0,26.0,67.0,106,53.0
416,3,10.0,08X337,THE SCHOOL FOR INQUIRY AND SOCIAL JUSTICE,0.781,181.0,39.0,24.0,96.0,51,51.0
52,4,10.0,27Q137,M.S. 137 AMERICA'S SCHOOL OF HEROES,0.541,617.0,179.0,27.0,37.0,50,50.0
48,5,9.0,28Q217,J.H.S. 217 ROBERT A. VAN WYCK,0.628,533.0,175.0,31.0,55.0,53,47.7
59,6,9.0,15K088,J.H.S. 088 PETER ROUGET,0.719,466.0,163.0,39.0,71.0,53,47.7
65,7,10.0,27Q202,J.H.S. 202 ROBERT H. GODDARD,0.593,350.0,98.0,26.0,55.0,44,44.0
45,8,10.0,30Q230,I.S. 230,0.601,439.0,157.0,36.0,57.0,38,38.0
445,9,10.0,30Q010,I.S. 010 HORACE GREELEY,0.605,233.0,56.0,22.0,58.0,36,36.0
71,10,10.0,10X141,RIVERDALE / KINGSBRIDGE ACADEMY (MIDDLE SCHOOL...,0.418,249.0,75.0,15.0,66.0,34,34.0
